# Here's an explanation of the code:

Imports and Helper Functions:
=
The code imports necessary modules from the Crypto library for cryptographic operations.
generate_aes_key() creates a random AES key for encryption.
generate_rsa_key_pair() generates a pair of RSA keys (public and private).
AES Encryption and Decryption:
=
Encryption (aes_encrypt):
Reads the file to be encrypted.
Encrypts the file using AES in EAX mode (which provides both encryption and authentication).
Saves the encrypted file with additional metadata (nonce and tag) needed for decryption.
Decryption (aes_decrypt):
Reads the encrypted file and extracts the nonce, tag, and ciphertext.
Decrypts the file using AES with the extracted nonce and tag.
Saves the decrypted file.
RSA Encryption and Decryption:
=
Encryption (rsa_encrypt):
Reads the file to be encrypted.
Encrypts the file using RSA with the provided public key.
Saves the encrypted file.
Decryption (rsa_decrypt):
Reads the encrypted file.
Decrypts the file using RSA with the provided private key.
Saves the decrypted file.
Key Management:
=
Saving Keys (save_key):
Saves the provided key to a file.
Loading Keys (load_key):
Reads a key from a file.
Example Usage:
=
Generates an AES key and RSA key pair.
Saves the RSA keys to files.
Demonstrates file encryption and decryption using both AES and RSA methods.
Overall, the code provides functions for encrypting and decrypting files using AES and RSA algorithms, along with managing and storing cryptographic keys.

In [ ]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import hashlib
import base64
import os

# Helper function to generate a random AES key
def generate_aes_key():
    return get_random_bytes(16)

# Helper function to generate RSA key pair
def generate_rsa_key_pair():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

# AES Encryption
def aes_encrypt(file_path, key):
    cipher = AES.new(key, AES.MODE_EAX)
    with open(file_path, 'rb') as file:
        plaintext = file.read()
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    with open(file_path + '.enc', 'wb') as file:
        [ file.write(x) for x in (cipher.nonce, tag, ciphertext) ]

# AES Decryption
def aes_decrypt(encrypted_file_path, key):
    with open(encrypted_file_path, 'rb') as file:
        nonce, tag, ciphertext = [ file.read(x) for x in (16, 16, -1) ]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    with open(encrypted_file_path[:-4], 'wb') as file:
        file.write(plaintext)

# RSA Encryption
def rsa_encrypt(file_path, public_key):
    cipher = PKCS1_OAEP.new(RSA.import_key(public_key))
    with open(file_path, 'rb') as file:
        plaintext = file.read()
    ciphertext = cipher.encrypt(plaintext)
    with open(file_path + '.enc', 'wb') as file:
        file.write(ciphertext)

# RSA Decryption
def rsa_decrypt(encrypted_file_path, private_key):
    cipher = PKCS1_OAEP.new(RSA.import_key(private_key))
    with open(encrypted_file_path, 'rb') as file:
        ciphertext = file.read()
    plaintext = cipher.decrypt(ciphertext)
    with open(encrypted_file_path[:-4], 'wb') as file:
        file.write(plaintext)

# Key management functions
def save_key(key, file_path):
    with open(file_path, 'wb') as file:
        file.write(key)

def load_key(file_path):
    with open(file_path, 'rb') as file:
        key = file.read()
    return key

# Example usage
if __name__ == "__main__":
    # Generate AES key and RSA key pair
    aes_key = generate_aes_key()
    rsa_private_key, rsa_public_key = generate_rsa_key_pair()

    # Save RSA keys
    save_key(rsa_private_key, 'private.pem')
    save_key(rsa_public_key, 'public.pem')

    # Encrypt and decrypt a file using AES
    aes_encrypt('sample.txt', aes_key)
    aes_decrypt('sample.txt.enc', aes_key)

    # Encrypt and decrypt a file using RSA
    rsa_encrypt('sample.txt', rsa_public_key)
    rsa_decrypt('sample.txt.enc', rsa_private_key)